In [1]:
import pandas as pd
import unicodedata
import requests
from datetime import datetime as dt

from bs4 import BeautifulSoup as soup
from helpers import regression as rh
from helpers.file import load_from_file
from services import MySQLService

In [2]:
season = 2024
target_stat = "pts"

mysql = MySQLService()

In [3]:
model_info = load_from_file("../dist/betting", target_stat)

print("Model name:", model_info["name"])
print("Model features:", model_info["features"])
print("Model stats_columns:", model_info["stats_columns"])
print("Model confidence margin:", model_info["confidence_margin"])

Model name: GradientBoosting
Model features: ['pts_avg', 'opponent_avg_conceded_pts', 'days_since_last_game', 'fg_pct_avg', 'opponent_avg_conceded_fg_pct']
Model stats_columns: ['pts', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']
Model confidence margin: 4.777442459794849


In [4]:
def remove_accents(input_str):
    return ''.join(
        c for c in unicodedata.normalize('NFD', input_str)
        if unicodedata.category(c) != 'Mn'
    )

season_games_plyrs = mysql.execute_query(f"""
    SELECT g.id as game_id, g.date, g.season, g.is_playoff, g.winner, g.home_id, p.name as player_name, g.away_id, pg.team_id, pg.player_id, pg.minutes, pg.pts, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.tov, pg.pf, pg.plus_minus
    FROM player_games AS pg
        LEFT JOIN games as g on pg.game_id = g.id 
        LEFT JOIN players as p on pg.player_id = p.id
    WHERE g.season = 2024
    ORDER BY g.date ASC""")

teams_df = mysql.get_data("teams")

order_by_clause = "date ASC"
games_df = mysql.get_data("games", order_by_clause=order_by_clause)

stats_columns = ['pts','fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
    'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']

rh.build_avg_columns(season_games_plyrs, stats_columns)

rh.build_days_since_last_game_col(season_games_plyrs)

season_games_plyrs = rh.build_opponent_conceded_columns(season_games_plyrs, games_df, stats_columns)

season_games_plyrs["player_name"] = season_games_plyrs["player_name"].str.replace(" Jr.", "")
season_games_plyrs['player_name'] = season_games_plyrs['player_name'].apply(remove_accents)

display(season_games_plyrs)

c:\Users\caiog\Projects\Personal\NBA-Predictor\src\services\mysql.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resp = pd.read_sql_query(query, db)
100%|██████████| 18/18 [00:02<00:00,  7.19it/s]


,game_id,date,season,is_playoff,winner,home_id,player_name,away_id,team_id_x,player_id,...,opponent_avg_conceded_fta,opponent_avg_conceded_ft_pct,opponent_avg_conceded_oreb,opponent_avg_conceded_dreb,opponent_avg_conceded_reb,opponent_avg_conceded_ast,opponent_avg_conceded_stl,opponent_avg_conceded_blk,opponent_avg_conceded_tov,opponent_avg_conceded_pf
0,22400061,2024-10-22,2024,0,H,1610612738,Jordan Walsh,1610612752,1610612738,1641775,...,18.4,0.7741,10.6,29.7,40.3,28.1,6.0,6.1,12.0,21.5
1,22400062,2024-10-22,2024,0,H,1610612747,Dalton Knecht,1610612750,1610612747,1642261,...,19.7,0.8202,9.0,31.3,40.3,22.7,5.9,5.6,12.1,19.5
2,22400061,2024-10-22,2024,0,H,1610612738,Tyler Kolek,1610612752,1610612752,1642278,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
3,22400062,2024-10-22,2024,0,H,1610612747,Bronny James,1610612750,1610612747,1642355,...,19.7,0.8202,9.0,31.3,40.3,22.7,5.9,5.6,12.1,19.5
4,22400061,2024-10-22,2024,0,H,1610612738,Pacome Dadiet,1610612752,1610612752,1642359,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8120,22401230,2024-12-14,2024,0,H,1610612760,Steven Adams,1610612745,1610612745,203500,...,23.9,0.7504,11.2,33.5,44.7,23.9,6.8,5.5,18.6,17.7
8121,22401229,2024-12-14,2024,0,H,1610612749,Giannis Antetokounmpo,1610612737,1610612749,203507,...,21.4,0.7672,10.4,33.0,43.4,28.2,9.7,6.4,16.2,19.6
8122,22401229,2024-12-14,2024,0,H,1610612749,Clint Capela,1610612737,1610612737,203991,...,21.2,0.8550,10.7,31.1,41.8,24.8,8.9,4.1,12.5,21.1
8123,22401229,2024-12-14,2024,0,H,1610612749,Bogdan Bogdanovic,1610612737,1610612737,203992,...,21.2,0.8550,10.7,31.1,41.8,24.8,8.9,4.1,12.5,21.1


In [5]:
season_games_plyrs.columns

Index(['game_id', 'date', 'season', 'is_playoff', 'winner', 'home_id',
       'player_name', 'away_id', 'team_id_x', 'player_id', 'minutes', 'pts',
       'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
       'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'plus_minus', 'pts_avg', 'fgm_avg', 'fga_avg', 'fg_pct_avg', 'fg3m_avg',
       'fg3a_avg', 'fg3_pct_avg', 'ftm_avg', 'fta_avg', 'ft_pct_avg',
       'oreb_avg', 'dreb_avg', 'reb_avg', 'ast_avg', 'stl_avg', 'blk_avg',
       'tov_avg', 'pf_avg', 'days_since_last_game', 'opponent_team_id',
       'team_id_y', 'opponent_avg_conceded_pts', 'opponent_avg_conceded_fgm',
       'opponent_avg_conceded_fga', 'opponent_avg_conceded_fg_pct',
       'opponent_avg_conceded_fg3m', 'opponent_avg_conceded_fg3a',
       'opponent_avg_conceded_fg3_pct', 'opponent_avg_conceded_ftm',
       'opponent_avg_conceded_fta', 'opponent_avg_conceded_ft_pct',
       'opponent_avg_conceded_oreb', 'opponent_avg_conceded

In [6]:
def get_games_info():
    url = "https://www.rotowire.com/basketball/nba-lineups.php"
    result = requests.get(url)
    doc = soup(result.text, "html.parser")
    games = doc.find_all("div", {"class": "lineup__box"})
    
    for g in games:
        try:
            game_dict = {
                'season': season,
                'date': dt.now()
            }
            
            teams = [x.text for x in g.find_all("div", {"class": "lineup__abbr"})]
            if len(teams) == 0:
                continue

            game_dict["away_abbv"] = teams[0]
            game_dict["home_abbv"] = teams[1]

            home_team = teams_df.loc[teams_df['abbreviation'] == game_dict["home_abbv"]].iloc[0]
            away_team = teams_df.loc[teams_df['abbreviation'] == game_dict["away_abbv"]].iloc[0]
            
            game_dict["away_id"] = away_team["id"]
            game_dict["home_id"] = home_team["id"]

            away_lineup = g.find("ul", {"class": "lineup__list is-visit"})
            home_lineup = g.find("ul", {"class": "lineup__list is-home"})

            away_lineup = [x.text.split('\n')[-2] for x in away_lineup.find_all("li", {"class": "lineup__player"})][:5]
            home_lineup = [x.text.split('\n')[-2] for x in home_lineup.find_all("li", {"class": "lineup__player"})][:5]

            home_players_df = rh.build_next_games_df(home_lineup, home_team, away_team, season_games_plyrs, stats_columns, games_df)
            away_players_df = rh.build_next_games_df(away_lineup, away_team, home_team, season_games_plyrs, stats_columns, games_df)

            merged_df = pd.concat([home_players_df, away_players_df], ignore_index=True)

            merged_df['pred'] = model_info["model"].predict(merged_df[model_info["features"]])

            merged_df['lower_bound'] = (merged_df['pred'] - model_info["confidence_margin"]).round()
            merged_df['upper_bound'] = (merged_df['pred'] + model_info["confidence_margin"]).round()
            
            display(merged_df[["lineup_name", 'player_name', 'team_name', 'opp_name', 'pred', 'lower_bound', 'upper_bound']])
        except Exception as e:
            print(e)
            continue

In [7]:
get_games_info()

,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,T. Haliburton,Tyrese Haliburton,Indiana Pacers,New Orleans Pelicans,20.938298,16.0,26.0
1,A. Nembhard,Andrew Nembhard,Indiana Pacers,New Orleans Pelicans,8.322981,4.0,13.0
2,B. Mathurin,Bennedict Mathurin,Indiana Pacers,New Orleans Pelicans,16.180403,11.0,21.0
3,Pascal Siakam,Pascal Siakam,Indiana Pacers,New Orleans Pelicans,17.264842,12.0,22.0
4,Myles Turner,Myles Turner,Indiana Pacers,New Orleans Pelicans,11.899433,7.0,17.0
5,D. Murray,Dejounte Murray,New Orleans Pelicans,Indiana Pacers,18.220324,13.0,23.0
6,CJ McCollum,CJ McCollum,New Orleans Pelicans,Indiana Pacers,21.362849,17.0,26.0
7,Trey Murphy,Trey Murphy III,New Orleans Pelicans,Indiana Pacers,19.322505,15.0,24.0
8,Herbert Jones,Herbert Jones,New Orleans Pelicans,Indiana Pacers,9.959250,5.0,15.0
9,Yves Missi,Yves Missi,New Orleans Pelicans,Indiana Pacers,12.034157,7.0,17.0


,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,Jalen Suggs,Jalen Suggs,Orlando Magic,New York Knicks,19.135122,14.0,24.0
1,K. Caldwell-Pope,Kentavious Caldwell-Pope,Orlando Magic,New York Knicks,9.797053,5.0,15.0
2,T. da Silva,Tristan da Silva,Orlando Magic,New York Knicks,6.525146,2.0,11.0
3,W. Carter,Wendell Carter,Orlando Magic,New York Knicks,6.569077,2.0,11.0
4,Goga Bitadze,Goga Bitadze,Orlando Magic,New York Knicks,8.363796,4.0,13.0
5,Jalen Brunson,Jalen Brunson,New York Knicks,Orlando Magic,22.123220,17.0,27.0
6,Mikal Bridges,Mikal Bridges,New York Knicks,Orlando Magic,16.466247,12.0,21.0
7,Josh Hart,Josh Hart,New York Knicks,Orlando Magic,12.121489,7.0,17.0
8,OG Anunoby,OG Anunoby,New York Knicks,Orlando Magic,17.361517,13.0,22.0
9,K. Towns,Karl-Anthony Towns,New York Knicks,Orlando Magic,20.268031,15.0,25.0


,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,Jordan Poole,Jordan Poole,Washington Wizards,Boston Celtics,18.732369,14.0,24.0
1,C. Carrington,Carlton Carrington,Washington Wizards,Boston Celtics,8.419047,4.0,13.0
2,B. Coulibaly,Bilal Coulibaly,Washington Wizards,Boston Celtics,11.988519,7.0,17.0
3,J. Champagnie,Justin Champagnie,Washington Wizards,Boston Celtics,11.542510,7.0,16.0
4,A. Sarr,Alexandre Sarr,Washington Wizards,Boston Celtics,11.291828,7.0,16.0
5,Jrue Holiday,Jrue Holiday,Boston Celtics,Washington Wizards,13.364087,9.0,18.0
6,Derrick White,Derrick White,Boston Celtics,Washington Wizards,15.440147,11.0,20.0
7,Jaylen Brown,Jaylen Brown,Boston Celtics,Washington Wizards,22.016788,17.0,27.0
8,Jayson Tatum,Jayson Tatum,Boston Celtics,Washington Wizards,24.823999,20.0,30.0
9,K. Porzingis,Kristaps Porzingis,Boston Celtics,Washington Wizards,20.259092,15.0,25.0


,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,Chris Paul,Chris Paul,San Antonio Spurs,Minnesota Timberwolves,9.165655,4.0,14.0
1,S. Castle,Stephon Castle,San Antonio Spurs,Minnesota Timberwolves,16.065786,11.0,21.0
2,J. Champagnie,Julian Champagnie,San Antonio Spurs,Minnesota Timberwolves,15.453675,11.0,20.0
3,H. Barnes,Harrison Barnes,San Antonio Spurs,Minnesota Timberwolves,11.829227,7.0,17.0
4,V. Wembanyama,Victor Wembanyama,San Antonio Spurs,Minnesota Timberwolves,25.671206,21.0,30.0
5,Mike Conley,Mike Conley,Minnesota Timberwolves,San Antonio Spurs,10.483752,6.0,15.0
6,A. Edwards,Anthony Edwards,Minnesota Timberwolves,San Antonio Spurs,24.021554,19.0,29.0
7,J. McDaniels,Jaden McDaniels,Minnesota Timberwolves,San Antonio Spurs,11.082663,6.0,16.0
8,Julius Randle,Julius Randle,Minnesota Timberwolves,San Antonio Spurs,18.607606,14.0,23.0
9,Rudy Gobert,Rudy Gobert,Minnesota Timberwolves,San Antonio Spurs,9.529937,5.0,14.0


,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,Tyus Jones,Tyus Jones,Phoenix Suns,Portland Trail Blazers,13.956633,9.0,19.0
1,Devin Booker,Devin Booker,Phoenix Suns,Portland Trail Blazers,26.303133,22.0,31.0
2,B. Beal,Bradley Beal,Phoenix Suns,Portland Trail Blazers,16.776875,12.0,22.0
3,Kevin Durant,Kevin Durant,Phoenix Suns,Portland Trail Blazers,23.307138,19.0,28.0
4,Jusuf Nurkic,Jusuf Nurkic,Phoenix Suns,Portland Trail Blazers,9.300407,5.0,14.0
5,A. Simons,Anfernee Simons,Portland Trail Blazers,Phoenix Suns,18.328058,14.0,23.0
6,S. Sharpe,Shaedon Sharpe,Portland Trail Blazers,Phoenix Suns,16.715823,12.0,21.0
7,T. Camara,Toumani Camara,Portland Trail Blazers,Phoenix Suns,9.213907,4.0,14.0
8,Jerami Grant,Jerami Grant,Portland Trail Blazers,Phoenix Suns,14.796938,10.0,20.0
9,D. Clingan,Donovan Clingan,Portland Trail Blazers,Phoenix Suns,7.460214,3.0,12.0


,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,S. Curry,Stephen Curry,Golden State Warriors,Dallas Mavericks,22.031567,17.0,27.0
1,Buddy Hield,Buddy Hield,Golden State Warriors,Dallas Mavericks,12.157894,7.0,17.0
2,A. Wiggins,Andrew Wiggins,Golden State Warriors,Dallas Mavericks,18.391189,14.0,23.0
3,J. Kuminga,Jonathan Kuminga,Golden State Warriors,Dallas Mavericks,16.994250,12.0,22.0
4,D. Green,Draymond Green,Golden State Warriors,Dallas Mavericks,8.002652,3.0,13.0
5,Luka Doncic,Luka Doncic,Dallas Mavericks,Golden State Warriors,24.950870,20.0,30.0
6,Kyrie Irving,Kyrie Irving,Dallas Mavericks,Golden State Warriors,21.026389,16.0,26.0
7,Klay Thompson,Klay Thompson,Dallas Mavericks,Golden State Warriors,12.902654,8.0,18.0
8,P. Washington,P.J. Washington,Dallas Mavericks,Golden State Warriors,13.574352,9.0,18.0
9,Dereck Lively,Dereck Lively II,Dallas Mavericks,Golden State Warriors,7.503345,3.0,12.0


,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,Austin Reaves,Austin Reaves,Los Angeles Lakers,Memphis Grizzlies,15.729695,11.0,21.0
1,Max Christie,Max Christie,Los Angeles Lakers,Memphis Grizzlies,8.748756,4.0,14.0
2,Rui Hachimura,Rui Hachimura,Los Angeles Lakers,Memphis Grizzlies,10.807156,6.0,16.0
3,L. James,LeBron James,Los Angeles Lakers,Memphis Grizzlies,21.134118,16.0,26.0
4,A. Davis,Anthony Davis,Los Angeles Lakers,Memphis Grizzlies,21.711247,17.0,26.0
5,Ja Morant,Ja Morant,Memphis Grizzlies,Los Angeles Lakers,21.428651,17.0,26.0
6,Desmond Bane,Desmond Bane,Memphis Grizzlies,Los Angeles Lakers,14.102318,9.0,19.0
7,Jaylen Wells,Jaylen Wells,Memphis Grizzlies,Los Angeles Lakers,11.168474,6.0,16.0
8,Jaren Jackson,Jaren Jackson,Memphis Grizzlies,Los Angeles Lakers,18.638572,14.0,23.0
9,B. Clarke,Brandon Clarke,Memphis Grizzlies,Los Angeles Lakers,9.339917,5.0,14.0
